In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize

In [7]:
client = MongoClient ('localhost', 27017)
db = client['companies_clean']
    
nearLocation = {
    "lng": 40.717432,
    "lat": -73.994943
}

comps = pd.DataFrame(db.comps.find({
    "coord": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ nearLocation["lat"] , nearLocation["lng"] ]
       },
       "$maxDistance": 5000, # In meters
     }
   }
}))

len(comps)

158